In [18]:
import pandas as pd
import boto3
import pyarrow
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.live import StockDataStream
from alpaca.data.requests import StockBarsRequest, StockTradesRequest
from alpaca.data.timeframe import TimeFrame

from dotenv import load_dotenv
import os
import json
from datetime import datetime, timedelta

In [19]:
load_dotenv()
ALPACA_API_KEY = os.environ.get("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.environ.get("ALPACA_SECRET_KEY")

## Reading Tickers

In [20]:
nasdaq = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nasdaq100.csv")
nyse = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nyse100.csv")

In [21]:
nasdaq.head()

,Ticker,Code,Name
0,FOXA,GOOG/NASDAQ_FOXA,21st Century Fox Class A
1,ATVI,GOOG/NASDAQ_ATVI,Activision Blizzard Inc
2,ADBE,GOOG/NASDAQ_ADBE,Adobe Systems Inc
3,AKAM,GOOG/NASDAQ_AKAM,Akamai Technologies Inc
4,ALXN,GOOG/NASDAQ_ALXN,Alexion Pharmaceuticals Inc


In [22]:
nyse.head()

,Ticker,Code,Name
0,MMM,GOOG/NYSE_MMM,3M Co.
1,ABT,GOOG/NYSE_ABT,Abbott Laboratories
2,ACN,GOOG/NYSE_ACN,Accenture PLC Cl A
3,AGN,GOOG/NYSE_AGN,Allergan Inc.
4,MO,GOOG/NYSE_MO,Altria Group Inc.


In [23]:
nasdaq_tickers = list(nasdaq.iloc[:, 0])
nyse_tickers = list(nyse.iloc[:, 0])
tickers = set(nasdaq_tickers + nyse_tickers)

## Batch Data Ingestion

In [24]:
hist_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

In [59]:
def convert_datetime(date: datetime):
    return datetime(date.year, date.month, date.day, 0, 0, 0)

def getStockHistoricalData(client: StockHistoricalDataClient, start_date, end_date = datetime.today()):
    bar_request = StockBarsRequest(
        symbol_or_symbols=tickers,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )

    # res = client.get_stock_bars(bar_request)
    res = client.get_stock_bars(bar_request)
    return(res)

bars = getStockHistoricalData(hist_client, datetime(2023, 1, 1), datetime(2023, 9, 8)).data

In [62]:
len(bars['ACN'])

171

In [63]:
BAR_SCHEMA = {
    "symbol": str,
    "high": "float64",
    "low": "float64",
    "open": "float64",
    "timestamp": "datetime64[ns]",
    "trade_count": "float64",
    "volume": "float64",
    "vwap": "float64"
}
df = pd.DataFrame(columns=BAR_SCHEMA.keys()).astype(BAR_SCHEMA)

for i, ticker in enumerate(bars):
    for bar in bars[ticker]:
        entry = {
            "symbol": bar.symbol,
            "high": bar.high,
            "low": bar.low,
            "open": bar.open,
            "timestamp": bar.timestamp,
            "trade_count": bar.trade_count,
            "volume": bar.volume,
            "vwap": bar.vwap
        }
        df.loc[len(df)] = entry


In [13]:
df.head()

,symbol,high,low,open,timestamp,trade_count,volume,vwap
0,AVGO,860.830,850.120,855.00,2023-09-08 04:00:00+00:00,56082.0,1681900.0,856.742460
1,HAL,41.640,41.050,41.11,2023-09-08 04:00:00+00:00,66518.0,9034302.0,41.428384
2,MDT,80.090,79.415,79.57,2023-09-08 04:00:00+00:00,43243.0,4208713.0,79.812639
3,ADSK,221.915,217.860,220.56,2023-09-08 04:00:00+00:00,21838.0,1002278.0,219.432816
4,AXP,157.790,156.710,157.02,2023-09-08 04:00:00+00:00,41938.0,1931613.0,157.404799


In [124]:
df.to_parquet("stocks.parquet.gzip", engine = "pyarrow", compression = "gzip")

## Sending to S3

In [125]:
s3 = boto3.client("s3")

file_path = "stocks.parquet.gzip"
bucket_name = "s3-alpaca-stock-data"


In [126]:
s3.upload_file(file_path, bucket_name, "historical/test.parquet.gzip")